# Benchmarking propositionalization approaches for predicting air pollution

**NOTE: Due to featuretools's and tsfresh's memory requirements, this notebook will not run on MyBinder.**

In this notebook we will compare getML to featuretools and tsfresh, both of which open-source libraries for feature engineering. We find that advanced algorithms featured in getML yield significantly better predictions on this dataset. We then discuss why that is.

Summary:

- Prediction type: __Regression model__
- Domain: __Air pollution__
- Prediction target: __pm 2.5 concentration__ 
- Source data: __Multivariate time series__
- Population size: __41757__

*Author: Dr. Patrick Urbanke*

## Background

Many data scientists and AutoML tools use propositionalization methods for feature engineering. These propositionalization methods usually work as follows:

- Generate a large number of more-or-less hard-coded features
- Use feature selection to pick a percentage of these features

getML's `FastPropModel` is an implementation of a propositionalization approach that has been optimized for speed and memory efficiency.

In this notebook, we will benchmark `FastPropModel` (https://getml.com/product) against featuretools (https://www.featuretools.com/) and tsfresh (https://tsfresh.readthedocs.io/en/latest/). Both of these libaries use propositionalization approaches for feature engineering.

As our example dataset, we use a publicly available dataset on air pollution in Beijing, China (https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data). The data set has been originally used in the following study:

Liang, X., Zou, T., Guo, B., Li, S., Zhang, H., Zhang, S., Huang, H. and Chen, S. X. (2015). Assessing Beijing's PM2.5 pollution: severity, weather impact, APEC and winter heating. Proceedings of the Royal Society A, 471, 20150257.

## A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

In [1]:
import datetime
import os
from urllib import request
import time

import getml
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
from utils import FTTimeSeriesBuilder
from utils import TSFreshBuilder

## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the UCI Machine Learning repository.

In [3]:
FEATURETOOLS_FILES = [
    "featuretools_training.csv",
    "featuretools_test.csv"
]

for fname in FEATURETOOLS_FILES:
    if not os.path.exists(fname):
        fname, res = request.urlretrieve(
            "https://static.getml.com/datasets/air_pollution/featuretools/" + fname, 
            fname
        )

In [4]:
TSFRESH_FILES = [
    "tsfresh_training.csv",
    "tsfresh_test.csv"
]

for fname in TSFRESH_FILES:
    if not os.path.exists(fname):
        fname, res = request.urlretrieve(
            "https://static.getml.com/datasets/air_pollution/tsfresh/" + fname, 
            fname
        )

In [5]:
fname = "PRSA_data_2010.1.1-2014.12.31.csv"

if not os.path.exists(fname):
    fname, res = request.urlretrieve(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00381/" + fname, 
        fname
    )


### 1.2 Prepare data for tsfresh and getML

Our our goal is to predict the pm2.5 concentration from factors such as weather or time of day. However, there are some **missing entries** for pm2.5, so we get rid of them.

In [6]:
data_full_pandas = pd.read_csv(fname)

data_full_pandas = data_full_pandas[
    data_full_pandas["pm2.5"] == data_full_pandas["pm2.5"]
]

tsfresh requires a date column, so we build one.

In [7]:
def add_leading_zero(val):
    if len(str(val)) == 1:
        return "0" + str(val)
    return str(val)

data_full_pandas["month"] = [
    add_leading_zero(val) for val in data_full_pandas["month"]
]

data_full_pandas["day"] = [
    add_leading_zero(val) for val in data_full_pandas["day"]
]

data_full_pandas["hour"] = [
    add_leading_zero(val) for val in data_full_pandas["hour"]
]

def make_date(year, month, day, hour):
    return year + "-" + month + "-" + day + " " + hour + ":00:00"

data_full_pandas["date"] = [
    make_date(str(year), month, day, hour) \
    for year, month, day, hour in zip(
        data_full_pandas["year"],
        data_full_pandas["month"],
        data_full_pandas["day"],
        data_full_pandas["hour"],
    )
]


tsfresh also requires the time series to have ids. Since there is only a single time series, that series has the same id.

In [8]:
data_full_pandas["id"] = 1

The dataset now contains many columns that we do not need or that tsfresh cannot process. For instance, *cbwd* might actually contain useful information, but it is a categorical variable, which is difficult to handle for tsfresh, so we remove it.

We also want to split our data into a training and testing set.

In [9]:
data_train_pandas = data_full_pandas[data_full_pandas["year"] < 2014]
data_test_pandas = data_full_pandas[data_full_pandas["year"] == 2014]
data_full_pandas = data_full_pandas

In [10]:
def remove_unwanted_columns(df):
    del df["cbwd"]
    del df["year"]
    del df["month"]
    del df["day"]
    del df["hour"]
    del df["No"]
    return df

data_full_pandas = remove_unwanted_columns(data_full_pandas)
data_train_pandas = remove_unwanted_columns(data_train_pandas)
data_test_pandas = remove_unwanted_columns(data_test_pandas)

In [11]:
data_full_pandas

,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,date,id
24,129.0,-16,-4.0,1020.0,1.79,0,0,2010-01-02 00:00:00,1
25,148.0,-15,-4.0,1020.0,2.68,0,0,2010-01-02 01:00:00,1
26,159.0,-11,-5.0,1021.0,3.57,0,0,2010-01-02 02:00:00,1
27,181.0,-7,-5.0,1022.0,5.36,1,0,2010-01-02 03:00:00,1
28,138.0,-7,-5.0,1022.0,6.25,2,0,2010-01-02 04:00:00,1
...,...,...,...,...,...,...,...,...,...
43819,8.0,-23,-2.0,1034.0,231.97,0,0,2014-12-31 19:00:00,1
43820,10.0,-22,-3.0,1034.0,237.78,0,0,2014-12-31 20:00:00,1
43821,10.0,-22,-3.0,1034.0,242.70,0,0,2014-12-31 21:00:00,1
43822,8.0,-22,-4.0,1034.0,246.72,0,0,2014-12-31 22:00:00,1


We then **load the data into the getML engine**. We begin by setting a project.

In [12]:
getml.engine.set_project('air_pollution')


Loading pipelines...
[========================================] 100%

Connected to project 'air_pollution'


In [13]:
df_full = getml.data.DataFrame.from_pandas(data_full_pandas, name='full')
df_train = getml.data.DataFrame.from_pandas(data_train_pandas, name='train')
df_test = getml.data.DataFrame.from_pandas(data_test_pandas, name='test')

df_full["date"] = df_full["date"].as_ts()

We need to **assign roles** to the columns, such as defining the target column.

In [14]:
def set_roles(df):
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role([
        "DEWP", 
        "TEMP",
        "PRES",
        "Iws",
        "Is",
        "Ir"], getml.data.roles.numerical)

set_roles(df_full)
set_roles(df_train)
set_roles(df_test)

## 2. Predictive modeling

In [15]:
population = getml.data.Placeholder('population')

peripheral = getml.data.Placeholder('peripheral')

population.join(
    peripheral,
    time_stamp='date',
    memory=getml.data.time.days(1)
)

population

placeholder   other placeholder   allow lagged targets   horizon   join keys used   memory    other join keys used   other time stamps used   relationship   time stamps used   upper time stamps used
  0   population    peripheral          False                  0.0                        86400.0                          date                     many-to-many   date

In [16]:
fast_prop = getml.feature_learning.FastPropModel(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    aggregation=getml.feature_learning.FastPropModel.agg_sets.All,
)

pipe_fp_fl = getml.pipeline.Pipeline(
    tags=['memory: 1d', 'simple features'],
    population=population,
    feature_learners=[fast_prop],)

pipe_fp_fl

Pipeline(preprocessors=[],
         feature_learners=['FastPropModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=[],
         tags=['memory: 1d', 'simple features'],
         share_selected_features=0.5)

In [17]:
pipe_fp_fl.check(df_train, [df_full])

Checking data model...
OK.


In [18]:
begin = time.time()

pipe_fp_fl.fit(df_train, [df_full])

fastprop_train = pipe_fp_fl.transform(
    df_train, 
    [df_full], 
    df_name="fastprop_train"
)

end = time.time()

fastprop_runtime = datetime.timedelta(seconds=end - begin)

Checking data model...
OK.

FastProp: Trying 226 features...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:18.003271


FastProp: Building features...
[========================================] 100%



In [19]:
fastprop_test = pipe_fp_fl.transform(df_test, [df_full], df_name="fastprop_test")


FastProp: Building features...
[========================================] 100%



In [20]:
predictor = getml.predictors.XGBoostRegressor()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [21]:
pipe_fp_pr.fit(fastprop_train)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:16.780831



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['prediction', 'fastprop'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/air_pollution/borJCm/0/

In [22]:
pipe_fp_pr.score(fastprop_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-16 10:41:33,fastprop_train,pm2.5,38.5582,55.3213,0.644
1,2021-05-16 10:41:33,fastprop_test,pm2.5,44.5867,63.5967,0.546


### 3.2 Using featuretools

To make things a bit easier, we have written a high-level wrapper around featuretools which we placed in a separate module (`utils`).

In [23]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(days=0),
    memory=pd.Timedelta(days=1),
    column_id="id",
    time_stamp="date",
    target="pm2.5")

In [24]:
featuretools_training = ft_builder.fit(data_train_pandas)
featuretools_test = ft_builder.transform(data_test_pandas)

featuretools: Trying features...


/home/patrick/.local/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Selecting the best out of 133 features...
Time taken: 0h:14m:39.009478



/home/patrick/.local/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [25]:
df_featuretools_training = getml.data.DataFrame.from_pandas(featuretools_training, name='featuretools_training')
df_featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test, name='featuretools_test')

In [26]:
def set_roles_featuretools(df):
    df["date"] = df["date"].as_ts()
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.unused_names, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df

df_featuretools_training = set_roles_featuretools(df_featuretools_training)
df_featuretools_test = set_roles_featuretools(df_featuretools_test)

In [27]:
predictor = getml.predictors.XGBoostRegressor()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=['featuretools', 'memory: 1d'],
    predictors=[predictor]
)

pipe_ft_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['featuretools', 'memory: 1d'],
         share_selected_features=0.5)

In [28]:
pipe_ft_pr.check(df_featuretools_training)

Checking data model...
OK.


In [29]:
pipe_ft_pr.fit(df_featuretools_training)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:11.506077



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['featuretools', 'memory: 1d'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/air_pollution/n7OYl0/0/

In [30]:
pipe_ft_pr.score(df_featuretools_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-16 11:00:24,featuretools_training,pm2.5,38.0455,54.4693,0.6567
1,2021-05-16 11:00:24,featuretools_test,pm2.5,45.3084,64.2717,0.5373


### 3.3 Using tsfresh

tsfresh is a rather low-level library. To make things a bit easier, we have written a high-level wrapper which we placed in a separate module (`utils`).

To limit the memory consumption, we undertake the following steps:

- We limit ourselves to a memory of 1 day from any point in time. This is necessary, because tsfresh duplicates records for every time stamp. That means that looking back 7 days instead of one day, the memory consumption would be  seven times as high.
- We extract only tsfresh's **MinimalFCParameters** and **IndexBasedFCParameters** (the latter is a superset of **TimeBasedFCParameters**).

In order to make sure that tsfresh's features can be compared to getML's features, we also do the following:

- We apply tsfresh's built-in feature selection algorithm.
- Of the remaining features, we only keep the 40 features most correlated with the target (in terms of the absolute value of the correlation).
- We add the original columns as additional features.


In [31]:
data_train_pandas

,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,date,id
24,129.0,-16,-4.0,1020.0,1.79,0,0,2010-01-02 00:00:00,1
25,148.0,-15,-4.0,1020.0,2.68,0,0,2010-01-02 01:00:00,1
26,159.0,-11,-5.0,1021.0,3.57,0,0,2010-01-02 02:00:00,1
27,181.0,-7,-5.0,1022.0,5.36,1,0,2010-01-02 03:00:00,1
28,138.0,-7,-5.0,1022.0,6.25,2,0,2010-01-02 04:00:00,1
...,...,...,...,...,...,...,...,...,...
35059,22.0,-19,7.0,1013.0,114.87,0,0,2013-12-31 19:00:00,1
35060,18.0,-21,7.0,1014.0,119.79,0,0,2013-12-31 20:00:00,1
35061,23.0,-21,7.0,1014.0,125.60,0,0,2013-12-31 21:00:00,1
35062,20.0,-21,6.0,1014.0,130.52,0,0,2013-12-31 22:00:00,1


One of the issues about tsfresh is that is actually requires more memory than allowed by MyBinder. We therefore have to remove the parts that relate to this.

In [32]:
tsfresh_builder = TSFreshBuilder(
    num_features=200,
    memory=24,
    column_id="id",
    time_stamp="date",
    target="pm2.5")

tsfresh_training = tsfresh_builder.fit(data_train_pandas)
tsfresh_test = tsfresh_builder.transform(data_test_pandas)

Feature Extraction: 100%|██████████| 20/20 [01:32<00:00,  4.62s/it]


Selecting the best out of 66 features...
Time taken: 0h:4m:5.60373



Feature Extraction: 100%|██████████| 20/20 [00:23<00:00,  1.20s/it]


tsfresh does not contain built-in machine learning algorithms. In order to ensure a fair comparison, we use the exact same machine learning algorithm we have also used for getML: An XGBoost regressor with all hyperparameters set to their default value.

In order to do so, we load the tsfresh features into the getML engine.

In [33]:
df_tsfresh_training = getml.data.DataFrame.from_pandas(tsfresh_training, name='tsfresh_training')
df_tsfresh_test = getml.data.DataFrame.from_pandas(tsfresh_test, name='tsfresh_test')

As usual, we need to set roles:

In [34]:
def set_roles_tsfresh(df):
    df["date"] = df["date"].as_ts()
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.unused_names, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df

df_tsfresh_training = set_roles_tsfresh(df_tsfresh_training)
df_tsfresh_test = set_roles_tsfresh(df_tsfresh_test)

In this case, our pipeline is very simple. It only consists of a single XGBoostRegressor.

In [35]:
predictor = getml.predictors.XGBoostRegressor()

pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=['tsfresh', 'memory: 1d'],
    predictors=[predictor]
)

pipe_tsf_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['tsfresh', 'memory: 1d'],
         share_selected_features=0.5)

In [36]:
pipe_tsf_pr.check(df_tsfresh_training)

Checking data model...
OK.


In [37]:
pipe_tsf_pr.fit(df_tsfresh_training)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:8.629762



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['tsfresh', 'memory: 1d'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/air_pollution/1t03eT/0/

In [38]:
pipe_tsf_pr.score(df_tsfresh_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-16 11:05:39,tsfresh_training,pm2.5,40.7577,57.8644,0.6098
1,2021-05-16 11:05:39,tsfresh_test,pm2.5,46.4913,65.9863,0.5087


## 3. Comparison

In [39]:
num_features = dict(
    fastprop=226,
    featuretools=133,
    tsfresh=55,
)

runtime_per_feature = [
    fastprop_runtime / num_features['fastprop'],
    ft_builder.runtime / num_features['featuretools'],
    tsfresh_builder.runtime / num_features['tsfresh'],
]

features_per_second = [1.0/r.total_seconds() for r in runtime_per_feature]

speedup_per_feature = [r/runtime_per_feature[0] for r in runtime_per_feature]

comparison = pd.DataFrame(
    dict(
        runtime=[fastprop_runtime, ft_builder.runtime, tsfresh_builder.runtime],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        speedup=[1, ft_builder.runtime/fastprop_runtime, tsfresh_builder.runtime/fastprop_runtime],
        speedup_per_feature=speedup_per_feature,
        mae=[pipe_fp_pr.mae, pipe_ft_pr.mae, pipe_tsf_pr.mae],
        rmse=[pipe_fp_pr.rmse, pipe_ft_pr.rmse, pipe_tsf_pr.rmse],
        rsquared=[pipe_fp_pr.rsquared, pipe_ft_pr.rsquared, pipe_tsf_pr.rsquared],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

comparison

,runtime,num_features,features_per_second,speedup,speedup_per_feature,mae,rmse,rsquared
getML: FastProp,0 days 00:00:32.363776,226,6.983094,1.000000,1.000000,44.586666,63.596680,0.545994
featuretools,0 days 00:14:39.009478,133,0.151307,27.160288,46.151924,45.308428,64.271739,0.537296
tsfresh,0 days 00:04:05.603730,55,0.223938,7.588847,31.183160,46.491281,65.986329,0.508669


## 4. Conclusion

We have compared getML's feature learning algorithms to tsfresh's brute-force feature engineering approaches on a data set related to air pollution in China. We found that getML significantly outperforms featuretools and tsfresh. These results are consistent with the view that feature learning can yield significant improvements over simple propositionalization approaches.

However, there are other datasets on which simple propositionalization performs well. Our suggestion is therefore to think of algorithms like `FastProp` and `RelMT` as tools in a toolbox. If a simple tool like `FastProp` gets the job done, then use that. But when you need more advanced approaches, like `RelMT`, you should have them at your disposal as well.

You are encouraged to reproduce these results. You will need getML (https://getml.com/product) and tsfresh (https://tsfresh.readthedocs.io/en/latest/). You can download both for free. 